In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
import random
import time
from faker import Faker
from pyspark.sql.functions import *
from datetime import datetime, timedelta

In [3]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Module2Exercise") \
    .config("spark.driver.memory", "512m") \
    .config("spark.executor.memory", "512m") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/08 07:07:19 WARN Utils: Your hostname, Sanju, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/08 07:07:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/08 07:07:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/08 07:07:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/08 07:07:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
# about 10 row: Create a DataFrame people with columns: id, name, age, city.

people = [
    (1, "Alice", 30, "New York"),(2, "Bob", 25, "Los Angeles"),(3, "Charlie", 35, "Chicago"), (4, "David", 28, "Houston"),(5, "Eve", 22, "Phoenix"), (6, "Frank", 40, "Philadelphia"),(7, "Grace", 27, "San Antonio"),(8, "Hannah", 32, "San Diego"),(9, "Ian", 29, "Dallas"),(10, "Judy", 24, "San Jose")
] 

orders = [(1, 1001, 250.75, "2023-10-01"), (2, 1002, 125.50, "2023-10-02"), (3, 1003, 300.00, "2023-10-03"),
          (4, 1004, 450.25, "2023-10-04"), (5, 1005, 80.00, "2023-10-05"), (6, 1006, 600.40, "2023-10-06"),
          (7, 1007, 150.15, "2023-10-07"), (8, 1008, 220.30, "2023-10-08"), (9, 1009, 330.60, "2023-10-09"),
          (10, 1010, 410.80, "2023-10-10")]

df_people = spark.createDataFrame(people, ["id", "name", "age", "city"])
df_orders = spark.createDataFrame(orders, ["user_id", "order_id", "amount", "order_date"])

In [9]:
df_people.createOrReplaceTempView("people")
df_orders.createOrReplaceTempView("orders")

In [10]:
spark.sql("SELECT * from people where age >= 21 order by age desc").show()

+---+-------+---+------------+
| id|   name|age|        city|
+---+-------+---+------------+
|  6|  Frank| 40|Philadelphia|
|  3|Charlie| 35|     Chicago|
|  8| Hannah| 32|   San Diego|
|  1|  Alice| 30|    New York|
|  9|    Ian| 29|      Dallas|
|  4|  David| 28|     Houston|
|  7|  Grace| 27| San Antonio|
|  2|    Bob| 25| Los Angeles|
| 10|   Judy| 24|    San Jose|
|  5|    Eve| 22|     Phoenix|
+---+-------+---+------------+



In [ ]:
spark.sql("SELECT id, name, COUNT(order_id) as total_orders, SUM(amount) as total_amount " \
"from people p JOIN orders o ON p.id = o.user_id GROUP BY id, name " \
"ORDER BY total_amount DESC").show()

+---+-------+------------+------------+
| id|   name|total_orders|total_amount|
+---+-------+------------+------------+
|  6|  Frank|           1|       600.4|
|  4|  David|           1|      450.25|
| 10|   Judy|           1|       410.8|
|  9|    Ian|           1|       330.6|
|  3|Charlie|           1|       300.0|
|  1|  Alice|           1|      250.75|
|  8| Hannah|           1|       220.3|
|  7|  Grace|           1|      150.15|
|  2|    Bob|           1|       125.5|
|  5|    Eve|           1|        80.0|
+---+-------+------------+------------+

